In [ ]:
import pandas as pd
import numpy  as np
from math import sin, cos, pi, sqrt
import random
import time
import rtsvg
rt = rtsvg.RACETrack()

_n_paths_         = 100
_n_circles_       = 100
_radius_min_      = 20
_radius_max_      = 30
_min_circle_sep_  = 30
_half_sep_        = _min_circle_sep_/2.0   # Needs to be more than the _radius_inc_test_
_radius_inc_test_ = 4
_radius_start_    = _radius_inc_test_ + 1  # Needs to be more than the _radius_inc_test_ ... less than the _min_circle_sep_
_escape_px_       = 10                     # less than the _min_circle_sep_

def createCircleDataset(n_circles=_n_circles_, n_paths=_n_paths_, radius_min=_radius_min_, radius_max=_radius_max_, min_circle_sep=_min_circle_sep_, radius_inc_test=_radius_inc_test_):
    circle_geoms = []
    def circleOverlaps(cx, cy, r):
        for _geom_ in circle_geoms:
            dx, dy = _geom_[0] - cx, _geom_[1] - cy
            d      = sqrt(dx*dx+dy*dy)
            if d < (r + _geom_[2] + _min_circle_sep_): # at least 10 pixels apart...
                return True
        return False
    def findOpening():
        _max_attempts_ = 100
        attempts  = 0
        cx, cy, r = random.randint(radius_max+min_circle_sep, 600-radius_max-min_circle_sep), \
                    random.randint(radius_max+min_circle_sep, 400-radius_max-min_circle_sep), random.randint(radius_min,radius_max)
        while circleOverlaps(cx,cy,r) and attempts < _max_attempts_:
            cx, cy, r = random.randint(radius_max+min_circle_sep, 600-radius_max-min_circle_sep), \
                        random.randint(radius_max+min_circle_sep, 400-radius_max-min_circle_sep), random.randint(radius_min,radius_max)
            attempts += 1
        if attempts == _max_attempts_:
            return None
        return cx, cy, r

    # Randomize the circles
    for i in range(n_circles):
        to_unpack = findOpening()
        if to_unpack is not None:
            cx, cy, r = to_unpack
            circle_geoms.append((cx,cy,r))

    # Randomize the entry point
    c0         = random.randint(0, len(circle_geoms)-1)
    cx, cy, r  = circle_geoms[c0]
    a0         = random.random() * 2 * pi
    entry_pt   = (cx+(r+_radius_inc_test_+0.5)*cos(a0),cy+(r+_radius_inc_test_+0.5)*sin(a0),c0)
                
    # Randomize the exit points
    exit_pts = []
    for i in range(n_paths):
        c1 = random.randint(0,len(circle_geoms)-1)
        while c1 == c0:
            c1 = random.randint(0,len(circle_geoms)-1)
        cx, cy, r  = circle_geoms[c1]
        a1         = random.random() * 2 * pi
        exit_pts.append((cx+(r+radius_inc_test+0.5)*cos(a1),cy+(r+radius_inc_test+0.5)*sin(a1),c1))

    return entry_pt, exit_pts, circle_geoms

_entry_pt_,_exit_pts_,_circle_geoms_ = createCircleDataset()

svg = '<svg x="0" y="0" width="600" height="400"><rect x="0" y="0" width="600" height="400" fill="#ffffff" />'
# Render Cirlces
for _geom_ in _circle_geoms_:
    svg += f'<circle cx="{_geom_[0]}" cy="{_geom_[1]}" r="{_geom_[2]}" stroke="#404040" fill="#a0a0a0" fill-opacity="0.2" />'
# Render Entry Points
svg += f'<circle cx="{_entry_pt_[0]}" cy="{_entry_pt_[1]}" r="3" stroke="#00af00" fill="#00af00" />'
svg += f'<circle cx="{_entry_pt_[0]}" cy="{_entry_pt_[1]}" r="6" stroke="#000000" fill="none" />'
# Render Exit Points
for i in range(len(_exit_pts_)):
    _exit_  = _exit_pts_[i]
    _color_ = rt.co_mgr.getColor(i) 
    svg += f'<circle cx="{_exit_[0]}" cy="{_exit_[1]}" r="3" stroke="{_color_}" fill="{_color_}" />'
svg_base = svg
rt.displaySVG(svg_base + '</svg>')

In [ ]:
#
# circularPathRouter() - route exits to a single entry around circles
# - all points needs to be at least circle_radius + radius_inc_test + 1.0 from the circle centers...
#
def circularPathRouter(entry_pt,                  # (x,y,circle_i) -- where circle_i is the circle index from circle_geoms
                       exit_pts,                  # [(x,y,circle_i),(x,y,circle_i),(x,y,circle_i), ...] -- where circle_i is the circle index from circle_geoms
                       circle_geoms,              # [(cx,cy,r),(cx,cy,r), ...]
                       escape_px          = 5,    # length to push the exit points (and entry point) away from circle
                       min_circle_sep     = 30,   # minimum distance between circles
                       half_sep           = 15,   # needs to be more than the radius_inc_test ... half separation (but doesn't have to be)
                       radius_inc_test    = 4,    # for routing around circles, how much to test with
                       radius_start       = 5,    # needs to be more than the radius_inc_test ... less than the min_circle_sep
                       max_pts_per_node   = 50,   # maximum points per node for the xy quad tree
                       merge_distance_min = 5):   # minimum distance necessary to merge a path into an already exiting path
    # Calculate a path around the circle geometries
    def calculatePathAroundCircles(pts):
        def breakSegment(_segment_):
            if rt.segmentLength(_segment_) < 2.0:
                return _segment_
            for _geom_ in circle_geoms:
                _circle_plus_ = (_geom_[0], _geom_[1], _geom_[2]+radius_inc_test)
                _dist_, _inter_  = rt.segmentIntersectsCircle(_segment_,_circle_plus_)
                if _dist_ <= _circle_plus_[2]:
                    if _inter_[0] == _geom_[0] and _inter_[1] == _geom_[1]:
                        dx, dy   = _segment_[1][0] - _segment_[0][0], _segment_[1][1] - _segment_[0][1]
                        l        = sqrt(dx*dx+dy*dy)
                        dx,  dy  = dx/l, dy/l
                        pdx, pdy = -dy, dx 
                        return [(_segment_[0][0], _segment_[0][1]), (_geom_[0] + pdx*(_geom_[2]+half_sep), _geom_[1] + pdy*(_geom_[2]+half_sep)), (_segment_[1][0], _segment_[1][1])]
                    else:
                        dx, dy = _inter_[0] - _geom_[0], _inter_[1] - _geom_[1]
                        l      = sqrt(dx*dx+dy*dy)
                        dx, dy = dx/l, dy/l
                        return [(_segment_[0][0], _segment_[0][1]), (_geom_[0]  + dx*(_geom_[2]+half_sep), _geom_[1]  + dy*(_geom_[2]+half_sep)), (_segment_[1][0], _segment_[1][1])]
            return _segment_
        last_length = 0
        _segments_  = []
        for _pt_ in pts:
            _segments_.append(_pt_)
        while last_length != len(_segments_):
            last_length    = len(_segments_)
            _new_segments_ = []
            for i in range(len(_segments_)-1):
                _new_ = breakSegment([_segments_[i], _segments_[i+1]])
                if len(_new_) == 3:
                    _new_segments_.append(_new_[0])
                    _new_segments_.append(_new_[1])
                else:
                    _new_segments_.append(_new_[0])
            _new_segments_.append(_new_[-1])
            _segments_ = _new_segments_        
        return _segments_
    
    # Fix up the the entry and exit points...
    x_min,y_min,x_max,y_max = entry_pt[0],entry_pt[1],entry_pt[0],entry_pt[1]
    entries = []
    x0,y0,ci  = entry_pt
    uv        = rt.unitVector(((circle_geoms[ci][0],circle_geoms[ci][1]),(x0,y0)))
    x0s,y0s   = x0+uv[0]*escape_px, y0+uv[1]*escape_px
    for pt in exit_pts:
        x1,y1,ci  = pt        
        uv        = rt.unitVector(((circle_geoms[ci][0],circle_geoms[ci][1]),(x1,y1)))
        x1s,y1s   = x1+uv[0]*escape_px, y1+uv[1]*escape_px
        entries.append([(x0,y0), (x0s,y0s), (x1s,y1s), (x1,y1)])
        x_min,y_min,x_max,y_max = min(x_min,x1),min(y_min,y1),max(x_max,x1),max(y_max,y1)
        x_min,y_min,x_max,y_max = min(x_min,x1s),min(y_min,y1s),max(x_max,x1s),max(y_max,y1s)

    # XY Quad Tree
    xy_tree = rt.xyQuadTree((x_min-half_sep,y_min-half_sep,x_max+half_sep,y_max+half_sep), max_pts_per_node=max_pts_per_node)

    # Sort paths by length (longest first)
    exit_sorter = []
    for i in range(len(entries)):
        _entry_ = entries[i]
        l = rt.segmentLength((_entry_[0], _entry_[3]))
        exit_sorter.append((l,i))
    exit_sorter = sorted(exit_sorter)
    exit_sorter.reverse()

    # keep track of all of the final paths
    paths, merge_info = [], []
    for i in range(len(entries)):
        paths.append(entries[i])
        merge_info.append((-1,-1))

    # plot out the longest path
    i_longest        = exit_sorter[0][1]
    pts              = entries[i_longest]
    _path_           = calculatePathAroundCircles(pts)
    _path_smooth_    = rt.smoothSegments(rt.expandSegmentsIntoPiecewiseCurvedParts(_path_, amp=5.0, ampends=8.0, max_travel=1))
    _path_smooth_.reverse()
    paths[i_longest] = _path_smooth_
    for i in range(len(_path_smooth_)):
        pt = (_path_smooth_[i][0], _path_smooth_[i][1], i_longest, i)
        xy_tree.add([pt])

    # analyze the other paths
    for i in range(1,len(exit_sorter)):
        i_path        =  exit_sorter[i][1]
        pts           =  entries[i_path]
        _path_        =  calculatePathAroundCircles(pts)
        _path_smooth_ =  rt.smoothSegments(rt.expandSegmentsIntoPiecewiseCurvedParts(_path_, amp=5.0, ampends=8.0, max_travel=1))    
        # merge with existing path
        merged_flag   = False
        _path_merged_ =  [_path_smooth_[-1]]
        for j in range(len(_path_smooth_)-2, 2, -1): # only down to 2... because the stem will exist from the longest path created
            closest = xy_tree.closest((_path_smooth_[j][0],_path_smooth_[j][1]), n=1)
            _path_merged_.append(_path_smooth_[j])
            if closest[0][0] < merge_distance_min:
                _path_merged_.append((closest[0][1][0], closest[0][1][1]))
                merged_flag = True
                break
        # save the path off
        paths[i_path] = _path_merged_
        if merged_flag:
            merge_info[i_path] = (closest[0][1][2], closest[0][1][3]) # path index ... path point
        # update xy tree
        for j in range(len(_path_merged_)-3): # don't include the exit points (don't want merges with them...)
            pt = (_path_merged_[j][0], _path_merged_[j][1], i_path, j)
            xy_tree.add([pt])

    # return the merged paths            
    return paths, merge_info

_paths_, _merge_info_ = circularPathRouter(_entry_pt_,_exit_pts_,_circle_geoms_)

In [ ]:
def svgPathTag(_segments_, stroke='#000000', stroke_width=0.4):
    d = f'M {_segments_[0][0]} {_segments_[0][1]}'
    for i in range(1,len(_segments_)):
        d += f' L {_segments_[i][0]} {_segments_[i][1]}'
    return f'<path d="{d}" stroke="{stroke}" stroke-width="{stroke_width}" fill="none"/>'

_paths_svg_ = []
for i in range(len(_paths_)):
    _path_ = _paths_[i]
    _paths_svg_.append(svgPathTag(_path_, stroke=rt.co_mgr.getColor(i), stroke_width=1.0))
_merge_svg_ = []
for i in range(len(_merge_info_)):
    _info_ = _merge_info_[i]
    if _info_[0] != -1:
        xy = _paths_[_info_[0]][_info_[1]]
        _merge_svg_.append(f'<circle cx="{xy[0]}" cy="{xy[1]}" r="9.0" fill="none" stroke="{rt.co_mgr.getColor(i)}" stroke-width="2.0" />')
rt.tile([svg_base + ''.join(_paths_svg_) + ''.join(_merge_svg_) + '</svg>', svg_base + ''.join(_paths_svg_) + '</svg>'])

In [ ]:
def accumulatedPathVolumeSVG(paths, merge_info, stroke='#000000', min_width=0.5, max_width=4):
    svg = []
    # make a lookup of merge point info
    path_to_points = {}
    for _info_ in merge_info:
        path_i  = _info_[0]
        point_j = _info_[1]
        if path_i not in path_to_points.keys():
            path_to_points[path_i] = {}
        if point_j not in path_to_points[path_i].keys():
            path_to_points[path_i][point_j] = 0
        path_to_points[path_i][point_j] += 1
    # iterate through the paths
    for i in range(len(paths)):
        path     = paths[i]
        stroke_w = min_width
        d        = f'M {path[0][0]} {path[0][1]}'
        for j in range(1,len(path)): # range(len(path)-2,-1,-1):
            d += f' L {path[j][0]} {path[j][1]}'
            if i in path_to_points.keys():
                if j in path_to_points[i].keys():
                    svg.append(f'<path d="{d}" stroke-width="{stroke_w}" fill="none" stroke="{stroke}" />')
                    d         = f'M {path[j][0]} {path[j][1]}'
                    stroke_w  = min(stroke_w + min_width, max_width)
        svg.append(f'<path d="{d}" stroke-width="{stroke_w}" fill="none" stroke="{stroke}" />')
    return ''.join(svg)

rt.displaySVG(svg_base + accumulatedPathVolumeSVG(_paths_, _merge_info_) + '</svg>')

In [ ]:
ns, ps, ts = [], [], []
for n_paths in range(10,300,20):
    _entry_pt_,_exit_pts_,_circle_geoms_ = createCircleDataset(n_paths=n_paths)
    for quad_tree_pts in range(10,120,20):
        t0 = time.time()
        _paths_, _merge_info_ = circularPathRouter(_entry_pt_, _exit_pts_, _circle_geoms_, max_pts_per_node=quad_tree_pts)
        t1 = time.time()
        ns.append(n_paths), ps.append(quad_tree_pts), ts.append(t1-t0)

In [ ]:
df = pd.DataFrame({'n_paths':ns, 'max_pts':ps, 'time':ts})
rt.tile([rt.xy(df, x_field='n_paths', y_field='time', line_groupby_field='max_pts', color_by='max_pts', dot_size=None, line_groupby_w=3, w=512),
         rt.histogram(df, bin_by='max_pts', color_by='max_pts')])

'<svg width="640.0" height="256.0" x="0" y="0" xmlns="http://www.w3.org/2000/svg"><svg id="xy_57719" x="0.0"  y="0.0"  width="512" height="256" xmlns="http://www.w3.org/2000/svg"><rect width="511" height="255" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 239 L 17 3 L 507 3 L 507 239 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,232 52,210 87,202 122,188 157,173 192,163 227,121 262,119 297,126 332,121 367,99 402,118 437,53 472,3 507,77 " stroke="#36a111" stroke-width="3" fill="none" /><polyline points="17,236 52,225 87,219 122,215 157,208 192,203 227,181 262,182 297,186 332,187 367,174 402,176 437,157 472,134 507,154 " stroke="#73e796" stroke-width="3" fill="none" /><polyline points="17,237 52,224 87,219 122,215 157,209 192,204 227,183 262,187 297,192 332,189 367,178 402,179 437,162 472,146 507,157 " stroke="#bc985c" stroke-width="3" fill="none" /><polyline points="17,236 52,222 87,216 122,212 157,208 192,200 227,179 262,184 297,188 332,184 367,173 402,174 437,157 472,143 507,150 " stroke="#f2aeaf" stroke-width="3" fill="none" /><polyline points="17,235 52,220 87,211 122,209 157,205 192,196 227,172 262,178 297,183 332,178 367,165 402,168 437,151 472,135 507,140 " stroke="#34a86c" stroke-width="3" fill="none" /><polyline points="17,235 52,217 87,207 122,205 157,201 192,190 227,165 262,171 297,177 332,169 367,157 402,160 437,139 472,126 507,128 " stroke="#a7c4fb" stroke-width="3" fill="none" /><text x="17" text-anchor="start" y="253" font-family="Times" fill="#000000" font-size="12px">10</text><text x="507" text-anchor="end" y="253" font-family="Times" fill="#000000" font-size="12px">290</text><text x="262.0" text-anchor="middle" y="253" font-family="Times" fill="#000000" font-size="12px">n_paths</text><text x="13" text-anchor="start" y="237" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,237)">0.06</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">2.10</text><text x="13" text-anchor="middle" y="120.0" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,120.0)">time</text><rect width="511" height="256" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><svg id="histogram_48279" x="512.0"  y="0.0"  width="128" height="256" xmlns="http://www.w3.org/2000/svg"><rect width="127" height="255" x="0" y="0" fill="#ffffff" stroke="#ffffff" /><rect id="histogram_48279_encsvgid_10" width="114.0" height="14" x="0" y="0" fill="#4988b6" stroke="#4988b6"/><rect x="0" y="0" width="114.0" height="14" fill="#4988b6" /><rect x="0" y="0" width="114.0" height="14" fill="#36a111" /><rect id="histogram_48279_encsvgid_30" width="114.0" height="14" x="0" y="15" fill="#4988b6" stroke="#4988b6"/><rect x="0" y="15" width="114.0" height="14" fill="#4988b6" /><rect x="0" y="15" width="114.0" height="14" fill="#73e796" /><rect id="histogram_48279_encsvgid_50" width="114.0" height="14" x="0" y="30" fill="#4988b6" stroke="#4988b6"/><rect x="0" y="30" width="114.0" height="14" fill="#4988b6" /><rect x="0" y="30" width="114.0" height="14" fill="#bc985c" /><rect id="histogram_48279_encsvgid_70" width="114.0" height="14" x="0" y="45" fill="#4988b6" stroke="#4988b6"/><rect x="0" y="45" width="114.0" height="14" fill="#4988b6" /><rect x="0" y="45" width="114.0" height="14" fill="#f2aeaf" /><rect id="histogram_48279_encsvgid_90" width="114.0" height="14" x="0" y="60" fill="#4988b6" stroke="#4988b6"/><rect x="0" y="60" width="114.0" height="14" fill="#4988b6" /><rect x="0" y="60" width="114.0" height="14" fill="#34a86c" /><rect id="histogram_48279_encsvgid_110" width="114.0" height="14" x="0" y="75" fill="#4988b6" stroke="#4988b6"/><rect x="0" y="75" width="114.0" height="14" fill="#4988b6" /><rect x="0" y="75" width="114.0" height="14" fill="#a7c4fb" /><text x="2" text-anchor="start" y="13" font-family="Times" fill="#000000" font-size="12px">10</text><text x="2" text-anchor="start" y="28" font-family="Times" fill="#000000" font-size="12px">30</text><text x="2" text-anchor="start" y="43" font-family="Times" fill="#000000" font-size="12px">50</text><text x="2" text-anchor="start" y="58" font-family="Times" fill="#000000" font-size="12px">70</text><text x="2" text-anchor="start" y="73" font-family="Times" fill="#000000" font-size="12px">90</text><text x="2" text-anchor="start" y="88" font-family="Times" fill="#000000" font-size="12px">110</text><text x="109" text-anchor="end" y="253" font-family="Times" fill="#000000" font-size="12px">15</text><line x1="114" y1="2" x2="114" y2="256" stroke="#101010" stroke-width="1" stroke-dasharray="3 2" /><text x="118" text-anchor="middle" y="128.0" font-family="Times" fill="#000000" font-size="12px" transform="rotate(90,118,128.0)">max_pts</text><rect width="127" height="255" x="0" y="0" fill-opacity="0.0" stroke="#000000" /></svg></svg>'